In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

%config Completer.use_jedi = False  # 자동완성 속도 증가

# 내장된 훈련 루프와 평가 루프 사용하기

In [2]:
# 표준 워크플로: compile(), fit(), evaluate(), predict()

#  모델 생성(재사용을 위해 함수로 제작)
def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model


# 데이터를 로드하고 검증을 위해 일부를 떼어놓음
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()

#  옵티마이저, 최소화할 손실 함수, 모니터링할 지표 지정
model.compile(
    optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# fit() 메서드를 사용하여 모델 훈련, 검증데이터를 함께 제공
model.fit(
    train_images, train_labels, epochs=3, validation_data=(val_images, val_labels)
)

# evaluate() 메서드를 사용하여 새로운 데이터에 대한 손실과 측정 지표를 계산
test_metrics = model.evaluate(test_images, test_labels)

# predict() 메서드를 사용하여 새로운 데이터에 대한 분류 확률을 계산
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2973 - accuracy: 0.9119 - val_loss: 0.1456 - val_accuracy: 0.9565
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1656 - accuracy: 0.9539 - val_loss: 0.1189 - val_accuracy: 0.9665
Epoch 3/3
313/313 [==============================] - 0s 1ms/step - loss: 0.1097 - accuracy: 0.9733


## 사용자 정의 지표 만들기

In [3]:
import tensorflow as tf


class RootMeanSquaredError(keras.metrics.Metric):  # metric 클래스 상속
    #  생성자에서 상태변수 정의
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )

    #  update_state() 메서드 안에 상태업데이트 로직 구현
    def update_state(self, y_true, y_pred, sample_weight=None):
        #  MNIST 모델의 예측은 각 클래스 확률이므로 원핫 인코딩
        #  y_true : 배치의 타깃, y_pred : 모델의 예측
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    #  현재 지표값 반환
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    #  상태 초기화
    def reset_state(self):
        self.mse_sum.assign(0.0)

In [4]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", RootMeanSquaredError()],
)
model.fit(
    train_images, train_labels, epochs=3, validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 6s 3ms/step - loss: 0.2940 - accuracy: 0.9128 - rmse: 7.1854 - val_loss: 0.1548 - val_accuracy: 0.9568 - val_rmse: 3.0019
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1646 - accuracy: 0.9537 - rmse: 4.9599 - val_loss: 0.1317 - val_accuracy: 0.9650 - val_rmse: 2.1362
Epoch 3/3
313/313 [==============================] - 0s 1ms/step - loss: 0.1235 - accuracy: 0.9674 - rmse: 1.7042


In [5]:
callbacks_list = [  # 콜백의 리스트를 모델로 전달
    keras.callbacks.EarlyStopping(  # 성능 향상이 멈추면 훈련 중지
        monitor="val_accuracy",  # 검증 정확도 모니터링
        patience=2,  # 두번의 에포크 동안 정확도 미향상시 훈련 중지
    ),
    keras.callbacks.ModelCheckpoint(  # 매 에포크 끝에 현재 가중치 저장
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
    ),
]
model = get_mnist_model()
model.compile(
    optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    train_images,
    train_labels,
    epochs=10,
    callbacks=callbacks_list,
    validation_data=(val_images, val_labels),
)
# model = keras.models.load_model('checkpoint_path.keras')

Epoch 1/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2946 - accuracy: 0.9112 - val_loss: 0.1562 - val_accuracy: 0.9532
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1624 - accuracy: 0.9544 - val_loss: 0.1230 - val_accuracy: 0.9673
Epoch 3/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1390 - accuracy: 0.9636 - val_loss: 0.1149 - val_accuracy: 0.9723
Epoch 4/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1247 - accuracy: 0.9674 - val_loss: 0.1117 - val_accuracy: 0.9717
Epoch 5/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1154 - accuracy: 0.9714 - val_loss: 0.1193 - val_accuracy: 0.9729
Epoch 6/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1101 - accuracy: 0.9733 - val_loss: 0.1109 - val_accuracy: 0.9759
Epoch 7/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1070 - accuracy: 0.9733 - val_loss: 0.1145 - val_accuracy: